In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Import the Libraries**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [ ]:
dir1 = ('../input/brain-tumor-mri-dataset/Training')
category = []


for file in os.listdir(dir1):
    category.append(file)
    
    
for i in category:
    file_path = os.path.join(dir1,i)
    
    for file in os.listdir(file_path):
        img_path = os.path.join(file_path,file)
        image = cv2.imread(img_path)
    

In [ ]:
plt.imshow(image)
print(image.shape)

**Lets see how image for each of the categories looks like**

In [ ]:
plt.figure(figsize=(15,8))

img_path = ['/pituitary/Tr-piTr_0001.jpg','/notumor/Tr-noTr_0001.jpg','/meningioma/Tr-meTr_0001.jpg','/glioma/Tr-glTr_0001.jpg']

for i in range(4):
    ax = plt.subplot(2, 2, i + 1)
    img = cv2.imread(dir1 + img_path[i])
    img = cv2.resize(img, (224, 224))
    plt.imshow(img)
    plt.title(category[i])

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

**We will us the ImageDataGenerator module for data augmentation**

*Lets understand why*

*Data augmentation is useful to improve performance and outcomes of models by forming new and different examples to train datasets.If the dataset in a machine learning model is rich and sufficient, the model performs better and more accurately*

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                    featurewise_center=False,
                                    samplewise_center=False,
                                    featurewise_std_normalization=False,
                                    samplewise_std_normalization=False,
                                    zca_whitening=False,
                                    rotation_range=0.2,
                                    zoom_range = 0.3,
                                    width_shift_range=0.2,
                                    height_shift_range=0,
                                    horizontal_flip=True,
                                    vertical_flip=False)


test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train = train_datagen.flow_from_directory('../input/brain-tumor-mri-dataset/Training',target_size=(200,200),class_mode='categorical',batch_size=32,color_mode='grayscale')
test=test_datagen.flow_from_directory('../input/brain-tumor-mri-dataset/Testing',target_size=(200,200),class_mode='categorical',batch_size=32,color_mode='grayscale')

**So now lets import everything that we need to create our CNN model**

In [ ]:
from tensorflow.keras import models,layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout
from tensorflow.keras.activations import relu,softmax
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping


**We are doing Batch normalization here but why ?**


*Distribution of the inputs to layers deep in the network may change after each mini-batch when the weights are updated. This can cause the learning algorithm to forever chase a moving target*

*Batch normalization standardizes the inputs to a layer for each mini-batch. This has the effect of stabilizing the learning process and dramatically reducing the number of training epochs required to train deep networks.*

In [ ]:
model = Sequential()

#layer 1
model.add(Conv2D(64,(7,7), input_shape=(200, 200, 1), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

#layer 2
model.add(Conv2D(128,(7,7), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

#layer 3
model.add(Conv2D(128,(7,7), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

#layer 4
model.add(Conv2D(256,(7,7), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

 #layer 5
model.add(Conv2D(256,(7,7), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

#layer 6
model.add(Conv2D(512,(7,7), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(units= 1024, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(units=512, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(units=4, activation='softmax'))



model.compile(optimizer='adam', loss='categorical_crossentropy',
                   metrics= ['categorical_accuracy'])

model.summary()


**Now its time to fit the model. we will use the concept of early stopping**

*Bfore we apply it we need to understand why*

*A problem with training neural networks is in the choice of the number of training epochs to use.

*Too many epochs can lead to overfitting of the training dataset, whereas too few may result in an underfit model*

*Early stopping alows you to specify an arbitrary large number of training epochs and stop training once the model performance stops improving on a hold out validation dataset*

In [ ]:
model_es = EarlyStopping(monitor = 'loss', min_delta = 1e-11, patience = 10, verbose = 1)

history = model.fit(train, epochs=100, validation_data=test,callbacks=model_es)

In [ ]:
model.save('Braintumormodel.h5')

In [ ]:
path='Weights_folder/Weights'
model.save_weights(path)

**Lets plot our accuracies and losses for both training and test**

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(15,8))
plt.plot(history.history["loss"],c = "blue")
plt.plot(history.history["val_loss"],c = "green")
plt.legend(["train", "test"])
plt.title('Loss')

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(15,8))
plt.plot(history.history["categorical_accuracy"],c = "blue")
plt.plot(history.history["val_categorical_accuracy"],c = "green")
plt.legend(["train", "test"])
plt.title('Accuracy')

**Ok now lets see if the model works properly**


**First we have to preprocess the image we will feed to the model**

In [ ]:
from tensorflow.keras.models import load_model

model1 = load_model('./Braintumormodel.h5')

In [ ]:
IMG_SIZE = 200
def prepare(filepath):
    img_array = cv2.imread(filepath)
    img_array = cv2.cvtColor(img_array,cv2.COLOR_BGR2GRAY)
    img_array = img_array / 255
    resized_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return resized_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)


**Now lets try to predict each class/category individually**

In [ ]:
def predict(prediction):
    if np.argmax(prediction) == 0:
        print('Glioma')
    elif np.argmax(prediction) == 1:
        print('Meningioma')   
    elif np.argmax(prediction) == 2:
        print('No tumor')    
    else:
        print('Pituatary')

**First one is Glioma**

In [ ]:
image = ('../input/brain-tumor-mri-dataset/Testing/glioma/Te-glTr_0008.jpg')
plt.imshow(plt.imread(image))
prediction = model1.predict([prepare(image)])
predict(prediction)


In [ ]:
image = ('../input/brain-tumor-mri-dataset/Testing/glioma/Te-gl_0027.jpg')
plt.imshow(plt.imread(image))
prediction = model.predict([prepare(image)])
predict(prediction)


**Next is Meningioma**

In [ ]:
image = ('../input/brain-tumor-mri-dataset/Testing/meningioma/Te-meTr_0000.jpg')
plt.imshow(plt.imread(image))
prediction = model.predict([prepare(image)])
predict(prediction)


In [ ]:
image = ('../input/brain-tumor-mri-dataset/Testing/meningioma/Te-me_0015.jpg')
plt.imshow(plt.imread(image))
prediction = model.predict([prepare(image)])
predict(prediction)

**Next one is No-Tumor**

In [ ]:
image = ('../input/brain-tumor-mri-dataset/Testing/notumor/Te-no_0029.jpg')
plt.imshow(plt.imread(image))
prediction = model.predict([prepare(image)])
predict(prediction)


In [ ]:
image = ('../input/brain-tumor-mri-dataset/Testing/notumor/Te-no_0017.jpg')
plt.imshow(plt.imread(image))
prediction = model.predict([prepare(image)])
predict(prediction)

**And the last one is Pituitary**

In [ ]:
image = ('../input/brain-tumor-mri-dataset/Testing/pituitary/Te-pi_0022.jpg')
plt.imshow(plt.imread(image))
prediction = model.predict([prepare(image)])
predict(prediction)

In [ ]:
image = ('../input/brain-tumor-mri-dataset/Testing/pituitary/Te-piTr_0007.jpg')
plt.imshow(plt.imread(image))
prediction = model.predict([prepare(image)])
predict(prediction)